# Truncated Singular Value Decomposition (TSVD) 
The TSVD algorithm is a linear dimensionality reduction algorithm that works really well for datasets in which samples correlated in large groups. Unlike PCA, TSVD does not center the data before computation. 

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames as the input. 

For information on converting your dataset to cuDF format, refer to the documentation: https://rapidsai.github.io/projects/cudf/en/0.11.0/

For information on cuML's TSVD implementation: https://rapidsai.github.io/projects/cuml/en/0.11.0/api.html#truncated-svd

In [ ]:
import os

import numpy as np

import pandas as pd
import cudf as gd

from cuml.datasets import make_blobs

from sklearn.decomposition import TruncatedSVD as skTSVD
from cuml.decomposition import TruncatedSVD as cumlTSVD

In [ ]:
n_samples = 2**15
n_features = 128

n_components = 2
random_state = 42

## Generate Data

### GPU

In [ ]:
device_data, _ = make_blobs(
   n_samples=n_samples, n_features=n_features, centers=1, random_state=7)

device_data = gd.DataFrame.from_gpu_matrix(device_data)

### Host

In [ ]:
host_data = device_data.to_pandas()

## Scikit-learn Model

In [ ]:
%%time
tsvd_sk = skTSVD(n_components=n_components,
                 algorithm="arpack", 
                 n_iter=5000,
                 tol=0.00001,
                 random_state=random_state)

result_sk = tsvd_sk.fit_transform(host_data)

## cuML Model

In [ ]:
%%time
tsvd_cuml = cumlTSVD(n_components=n_components,
                     algorithm="full", 
                     n_iter=50000,
                     tol=0.00001,
                     random_state=random_state)

result_cuml = tsvd_cuml.fit_transform(device_data)

## Evaluate Results

### Singular Values

In [ ]:
passed = np.allclose(tsvd_sk.singular_values_, 
                     tsvd_cuml.singular_values_.to_array(), 
                     atol=0.01)
print('compare tsvd: cuml vs sklearn singular_values_ {}'.format('equal' if passed else 'NOT equal'))

### Components

In [ ]:
passed = np.allclose(tsvd_sk.components_, 
                     np.asarray(tsvd_cuml.components_.as_gpu_matrix()), 
                     atol=1e-2)
print('compare tsvd: cuml vs sklearn components_ {}'.format('equal' if passed else 'NOT equal'))

### Transform

In [ ]:
# compare the reduced matrix
passed = np.allclose(result_sk, np.asarray(result_cuml.as_gpu_matrix()), atol=0.2)
# larger error margin due to different algorithms: arpack vs full
print('compare tsvd: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal'))